# CMS Data Exploration

Here we are examining the data from Centers for Medicare and Medicaid Services for 2019's general open payments system. Open payments reveal the nature of payments from companies to doctors and hospitals for the sake of transparency between healthcare professionals, drug companies, and the public. From the CMS website's summary information, we see that 1602 companies made 10.3 million general payments totalling 3.56 billion USD to 1196 teaching hospitals and 615k physicians.

Our query is how can a drug company optimize its spending? What inputs (physician name, type, hospital) determine the cost of a particular transaction? We will explore what the data contains and what it does not contain?

Additionally, once we have some insight regarding spending patterns of various companies, we might be able to use machine learning techniques to build a recommendation system: how can a company build a budget for a future year? Based on hospital / physician, is there a recommended expenditure?

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sklearn
from sklearn import preprocessing
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.tree import DecisionTreeRegressor
import category_encoders as ce
import datetime

## Loading Data

In [2]:
# Not specifying "Dtype"-argument yields: DtypeWarning:
# Columns 4,7,9,11,14,16,17,21,22,23,24,28,35,36,37,40,41,42,43,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
# have mixed types.Specify dtype option on import or set low_memory=False.
cms_gen_data = pd.read_csv('D:/Documents/Projects/Data_Sets/PGYR19_P063020/OP_DTL_GNRL_PGYR2019_P06302020.csv', low_memory=False)

In [3]:
cms_gen_data.head()

,Change_Type,Covered_Recipient_Type,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Physician_Profile_ID,Physician_First_Name,Physician_Middle_Name,Physician_Last_Name,Physician_Name_Suffix,...,Product_Category_or_Therapeutic_Area_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Associated_Drug_or_Biological_NDC_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,Associated_Drug_or_Biological_NDC_5,Program_Year,Payment_Publication_Date
0,NEW,Covered Recipient Physician,NaN,NaN,NaN,827477.0,STEPHEN,W.,BOATRIGHT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,06/30/2020
1,NEW,Covered Recipient Physician,NaN,NaN,NaN,623090.0,JASON,NaN,FOREMAN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,06/30/2020
2,NEW,Covered Recipient Physician,NaN,NaN,NaN,3362255.0,MAURICE,NaN,HAMAOUI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,06/30/2020
3,NEW,Covered Recipient Physician,NaN,NaN,NaN,3362255.0,MAURICE,NaN,HAMAOUI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,06/30/2020
4,NEW,Covered Recipient Physician,NaN,NaN,NaN,3362255.0,MAURICE,NaN,HAMAOUI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,06/30/2020


In [4]:
cms_gen_data.shape

(10367307, 75)

In [5]:
cms_gen_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10367307 entries, 0 to 10367306
Data columns (total 75 columns):
 #   Column                                                             Dtype  
---  ------                                                             -----  
 0   Change_Type                                                        object 
 1   Covered_Recipient_Type                                             object 
 2   Teaching_Hospital_CCN                                              float64
 3   Teaching_Hospital_ID                                               float64
 4   Teaching_Hospital_Name                                             object 
 5   Physician_Profile_ID                                               float64
 6   Physician_First_Name                                               object 
 7   Physician_Middle_Name                                              object 
 8   Physician_Last_Name                                                object 
 9   

In [6]:
# missing values?
missing = pd.concat([cms_gen_data.isnull().sum(), 100 * cms_gen_data.isnull().mean()], axis=1)
missing.columns=['count','%']
missing.sort_values(by='count')

,count,%
Change_Type,0,0.000000
Related_Product_Indicator,0,0.000000
Dispute_Status_for_Publication,0,0.000000
Record_ID,0,0.000000
Delay_in_Publication_Indicator,0,0.000000
...,...,...
Physician_License_State_code3,10333756,99.676377
Physician_License_State_code4,10360495,99.934293
Physician_License_State_code5,10366143,99.988772
Recipient_Postal_Code,10366845,99.995544


There are several ways to narrow down the data. For example within these 75 columns, a large number of them deal with different items relating to the recipient's info (name, address, etc), and perhaps it may not be too necessary to include information in our analysis such as which states a particular physician is licensed in (5 columns of licensing information). Perhaps it would serve us to just group the recipients by state and / or country instead of including the whole mailing address (Recipient_Postal_Code is a column that is 99.99% blank). Column 41 is a "Charity Indicator", either yes or no, but most records do not include this information, rendering it less hopeful. Columns 73 and 74 are all the same value - as long as we are only examining the data for 2019, the information in these columns is identical. Therefore this data will only be useful when we start comparing across different years later on.

The most important columns for us will be columns 30-34, which deal specifically with the money exchanges - total amounts, how many payments, the nature of the payments, etc. Our first goal will be to gain insight into determining factors for the payment amounts.

Columns 48-72 contain data regarding the products (services, drugs, devices) that the companies are trying to offer to doctors and hospitals. The data here may be useful in categorizing the payments.

Perhaps we may want to change the scope of our analysis in different ways too - maybe we only want to look at transactions within the US. 


## Exploratory Data Analysis

In [7]:
#Which companies are making the most number of general payments?
cms_gen_data['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'].value_counts()

PFIZER INC.                            507479
AstraZeneca Pharmaceuticals LP         413859
Amgen Inc.                             387662
Novo Nordisk Inc                       382542
Allergan Inc.                          381369
                                        ...  
Arjo Inc.                                   1
TrackX Technology, LLC                      1
Asahi Kasei Medical Co., Ltd.               1
Vertex Pharmaceuticals Incorporated         1
SCS Boehringer Ingelheim Comm.V             1
Name: Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name, Length: 1564, dtype: int64

In [8]:
cms_gen_data['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID'].value_counts()

100000000286    507479
100000000146    413859
100000000203    387662
100000000144    382542
100000000278    381369
                 ...  
100000136435         1
100000000334         1
100000046272         1
100000010748         1
100000011190         1
Name: Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID, Length: 1541, dtype: int64

In [9]:
cms_gen_data['Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name'].value_counts()

Pfizer Inc.                        511645
AstraZeneca Pharmaceuticals LP     417269
Amgen Inc.                         387662
Novo Nordisk Inc                   382569
Allergan Inc.                      381369
                                    ...  
Itamar Medical Inc                      1
Parsolex GMP Center, Inc.               1
B. Braun Miethke GmbH & Co. KG          1
Pacira CryoTech Incorporated            1
Fort Worth Surgical Supply, LLC         1
Name: Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name, Length: 1458, dtype: int64

It seems redundant to have the companies making payments classified by both name and ID, both of which are presumably unique identifiers. However, the number of different IDs (1541) is less than the number of names (1564). Perhaps a company might have filed with a different name, but in the open payment system they used the same ID.

In [10]:
#This will give us an idea of who is paying the most
company_payment_sum = cms_gen_data.groupby('Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name')[['Total_Amount_of_Payment_USDollars','Number_of_Payments_Included_in_Total_Amount']].sum()
company_payment_sum_desamount = company_payment_sum.sort_values('Total_Amount_of_Payment_USDollars',ascending=False)
company_payment_sum_desamount.head(20)

,Total_Amount_of_Payment_USDollars,Number_of_Payments_Included_in_Total_Amount
Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,,
"Genentech, Inc.",4.094500e+08,3991
"Takeda Pharmaceuticals U.S.A., Inc.",3.144871e+08,148644
"Zimmer Biomet Holdings, Inc.",3.038728e+08,89454
"DePuy Synthes Products, Inc.",1.354403e+08,2874
Stryker Corporation,8.924227e+07,124526
Boston Scientific Corporation,8.743868e+07,157602
"Medtronic USA, Inc.",8.248062e+07,94966
Allergan Inc.,7.971863e+07,381369
"Arthrex, Inc.",7.867567e+07,43234


In [11]:
#This will give us an idea of who is making the most number of payments
company_payment_sum_desnumber = company_payment_sum.sort_values('Number_of_Payments_Included_in_Total_Amount',ascending=False)
company_payment_sum_desnumber.head(20)

,Total_Amount_of_Payment_USDollars,Number_of_Payments_Included_in_Total_Amount
Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,,
AstraZeneca Pharmaceuticals LP,7.050708e+07,581775
PFIZER INC.,4.467429e+07,507479
Amgen Inc.,4.403313e+07,387662
Novo Nordisk Inc,2.423411e+07,382542
Allergan Inc.,7.971863e+07,381369
Novartis Pharmaceuticals Corporation,4.127666e+07,349784
"Janssen Pharmaceuticals, Inc",3.397790e+07,337029
"AbbVie, Inc.",5.822602e+07,329289
"Lilly USA, LLC",3.779572e+07,289264


In [12]:
#What are the nature of the general payments? The CMS categorizes open payments with a list of categories defined here:
#https://www.cms.gov/OpenPayments/About/Natures-of-Payment.
cms_gen_data['Nature_of_Payment_or_Transfer_of_Value'].value_counts()

Food and Beverage                                                                                                                                     9059333
Travel and Lodging                                                                                                                                     616217
Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program     243245
Education                                                                                                                                              191364
Consulting Fee                                                                                                                                         141341
Gift                                                                                                                                                    42587
Honoraria                                           

In [13]:
#Who are the recipients of the payments?
cms_gen_data['Covered_Recipient_Type'].value_counts()

Covered Recipient Physician            10319667
Covered Recipient Teaching Hospital       47640
Name: Covered_Recipient_Type, dtype: int64

The above shows that each payment is either to a teaching hospital or a physician. There are no other types of payments in this database, and there are no payments here that are unclassified.

In [14]:
cms_gen_data['Physician_Profile_ID'].value_counts()

57227.0      1249
232968.0     1226
324864.0      980
213579.0      921
354123.0      900
             ... 
321418.0        1
321417.0        1
1138073.0       1
1138076.0       1
4366476.0       1
Name: Physician_Profile_ID, Length: 613924, dtype: int64

In [15]:
# Which countries are involved in this general payments dataset?
cms_gen_data['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country'].value_counts()

United States                              10281957
Ireland                                       53248
Great Britain (Uk)                             7362
Switzerland                                    7039
Canada                                         3870
Denmark                                        2844
Barbados                                       1862
Germany                                        1664
Japan                                          1452
France                                         1374
Iceland                                        1367
Belgium                                        1203
Sweden                                          659
Israel                                          653
Italy                                           276
New Zealand                                      77
Australia                                        69
Austria                                          50
Korea (Republic of)                              48
Brazil      

In [16]:
cms_gen_data['Recipient_Country'].value_counts()

United States                           10366845
United States Minor Outlying Islands         176
Canada                                        99
Great Britain (Uk)                            48
Mexico                                        32
Germany                                       31
Korea (Republic of)                           12
United Arab Emirates                          12
Russian Federation                             7
Japan                                          7
Italy                                          6
Thailand                                       4
India                                          4
Saudi Arabia                                   3
Dominican Republic                             2
Guyana                                         2
Israel                                         2
Lebanon                                        2
Bahamas                                        1
Taiwan                                         1
Spain               

It may make the most sense to only deal with the USA since that includes more than 99% of the data, and some countries have only a single-digit number of transactions associated with them.

In [17]:
#Let's examine a big company in particular, to see exactly what they are paying
AZP = cms_gen_data[cms_gen_data['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'] == 'AstraZeneca Pharmaceuticals LP']
AZP.head()

,Change_Type,Covered_Recipient_Type,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Physician_Profile_ID,Physician_First_Name,Physician_Middle_Name,Physician_Last_Name,Physician_Name_Suffix,...,Product_Category_or_Therapeutic_Area_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Associated_Drug_or_Biological_NDC_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,Associated_Drug_or_Biological_NDC_5,Program_Year,Payment_Publication_Date
3186902,NEW,Covered Recipient Physician,NaN,NaN,NaN,78055.0,Burton,G,Shaw,JR.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,06/30/2020
3186903,NEW,Covered Recipient Physician,NaN,NaN,NaN,85409.0,Raja,G,Devanathan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,06/30/2020
3186904,NEW,Covered Recipient Physician,NaN,NaN,NaN,36873.0,Anthony,M,Cotronea,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,06/30/2020
3186905,NEW,Covered Recipient Physician,NaN,NaN,NaN,1304787.0,Juliet,Gyamfua,Appiah,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,06/30/2020
3186906,NEW,Covered Recipient Physician,NaN,NaN,NaN,2790146.0,Christopher,NaN,Dingess,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,06/30/2020


In [18]:
AZP_nature = AZP.groupby('Nature_of_Payment_or_Transfer_of_Value')[['Total_Amount_of_Payment_USDollars','Number_of_Payments_Included_in_Total_Amount']].sum()
AZP_nature

,Total_Amount_of_Payment_USDollars,Number_of_Payments_Included_in_Total_Amount
Nature_of_Payment_or_Transfer_of_Value,,
Charitable Contribution,7.604752e+04,60
"Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program",2.475140e+07,8841
Consulting Fee,9.709371e+06,2595
Education,1.127989e+05,10591
Food and Beverage,9.101708e+06,548296
Gift,1.000000e+04,1
Grant,5.189673e+05,72
Royalty or License,2.257419e+07,8
Space rental or facility fees (teaching hospital only),7.977722e+05,606


In [19]:
#Are there any records that do not have a transaction price? Those will be dropped, as they are not useful for the analysis
print(cms_gen_data['Total_Amount_of_Payment_USDollars'].isnull().sum())
print(cms_gen_data['Number_of_Payments_Included_in_Total_Amount'].isnull().sum())

0
0


Since no row lacks its payment price data, no rows have to be dropped for the preliminary development of a machine learning model for lack of payment price. Our end goal is to develop a tool that can predict the price of a payment, when its key determining factors are known. However, the determining factors of payment price are not currently established from the data. Some preliminary modeling will help us in that regard.

In terms of model selection, almost all of the features of the data are not numerical, but rather categorical such as type of payment, company name, or country / state of recipient, so the best models will take that into account. Decision Trees and Random Forest models can take categoricals into account, but in sklearn they must be encoded as numericals. This might prove to be too computationally expensive. We'll try simplifying the dataframe a little first (and justify our choices along the way). NaN values must also be dealt with.

## Data Wrangling

### Possible columns to delete: 
Any column that deals with information regarding the publishing of a payment as opposed to the nature of the payment itself can be dropped. The column "Change_Type" seems to act sort of like version control for each payment. "Payment Publication Date" also deals with when the payment information was published and doesn't give information regarding the payment itself either. Other columns seem to include: Delay_in_Publication_Indicator, Dispute_Status_for_Publication. record_ID serves as a unique identifier for each individual payment.

"Covered_Recipient_Type" is 100% populated, and classifies whether the payment recipient is a physician or hospital. A closer look at the individual recipients would show that certain physicians and certain hospitals do receive a large percentage of 2019's overall payment total. But perhaps the recipient information can be simplified. For example, instead of using names, only their IDs could be used. With both name information and unique IDs, the identifications are redundant. (Perhaps it may make sense to divide up the data to two different dataframes - one for hospital and one for physician - for human readability of the data.)

In regards to recipient data, we will also assume that the particular street address is irrelevant, but city and / or state may be valuable. However, perhaps this geographical information may be combined to further simplify things. There also exist payments to or from places outside the US, but those payments total to <1% of the monetary total and <1% of the total number of payments, so it may be expedient to just drop those records, which would also imply dropping columns about geographical data on places outside the US (province, country, postal code).

However, address can perhaps be used as a unique identifier for all recipients instead of two columns of IDs, assuming multiple recipients do not share the same address, and the same street address does not occur in multiple states. - Too many assumptions / things to check!?


In [23]:
#Dropping rows where recipients are located outside the US
cms_edit1 = cms_gen_data[cms_gen_data['Recipient_Country']=='United States']

In [24]:
#Dropping least relevant columns
cms_edit1 = cms_edit1.drop(
    ['Change_Type',
     'Teaching_Hospital_CCN',
     'Teaching_Hospital_Name',
     'Physician_First_Name',
     'Physician_Middle_Name',
     'Physician_Last_Name',
     'Physician_Name_Suffix',
     'Recipient_Primary_Business_Street_Address_Line1',
     'Recipient_Primary_Business_Street_Address_Line2',
     'Recipient_Zip_Code',
     'Recipient_Country',
     'Recipient_Province',
     'Recipient_Postal_Code',
     'Record_ID',
     'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
     'Contextual_Information',
     'Delay_in_Publication_Indicator',
     'Dispute_Status_for_Publication',
     'Payment_Publication_Date'],axis=1)

cms_edit1.head()

,Covered_Recipient_Type,Teaching_Hospital_ID,Physician_Profile_ID,Recipient_City,Recipient_State,Physician_Primary_Type,Physician_Specialty,Physician_License_State_code1,Physician_License_State_code2,Physician_License_State_code3,...,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4,Product_Category_or_Therapeutic_Area_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Associated_Drug_or_Biological_NDC_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,Associated_Drug_or_Biological_NDC_5,Program_Year
0,Covered Recipient Physician,NaN,827477.0,NORTH LITTLE ROCK,AR,Doctor of Dentistry,Dental Providers|Dentist|Endodontics,AR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1,Covered Recipient Physician,NaN,623090.0,CHAMPAIGN,IL,Doctor of Dentistry,Dental Providers|Dentist|Endodontics,IL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
2,Covered Recipient Physician,NaN,3362255.0,BROOKLYN,NY,Doctor of Dentistry,Dental Providers|Dentist,NY,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
3,Covered Recipient Physician,NaN,3362255.0,BROOKLYN,NY,Doctor of Dentistry,Dental Providers|Dentist,NY,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
4,Covered Recipient Physician,NaN,3362255.0,BROOKLYN,NY,Doctor of Dentistry,Dental Providers|Dentist,NY,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019


In [25]:
#Check: if hospital ID and physician ID are unique values
unique_physicians = cms_edit1['Physician_Profile_ID'].nunique()
unique_hospitals = cms_edit1['Teaching_Hospital_ID'].nunique()
print([unique_physicians,unique_hospitals])
print(unique_physicians + unique_hospitals)

[613876, 1172]
615048


In [26]:
#Making a new column "Recipient_ID" that will replace the two columns "Teaching_Hospital_ID" and "Physician_Profile_ID"
cms_edit1['Recipient_ID'] = cms_edit1['Teaching_Hospital_ID'].fillna(cms_edit1['Physician_Profile_ID'])

#Assuming all recipient IDs are unique, this value should equal the sum of the number of unique values of both columns
cms_edit1['Recipient_ID'].nunique()

614233

There are 815 IDs that seem to be shared between physician and hospital ID columns. If that is unintended, then adding a prefix/suffix to the IDs (such as 'h' for hospitals) can easily distinguish them, and solve this issue of non-uniqueness.

In [27]:
#Are there columns that contain both hospital and physician ID?
cms_edit1[(cms_edit1['Teaching_Hospital_ID'].notna()) & (cms_edit1['Physician_Profile_ID'].notna())]

,Covered_Recipient_Type,Teaching_Hospital_ID,Physician_Profile_ID,Recipient_City,Recipient_State,Physician_Primary_Type,Physician_Specialty,Physician_License_State_code1,Physician_License_State_code2,Physician_License_State_code3,...,Product_Category_or_Therapeutic_Area_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Associated_Drug_or_Biological_NDC_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,Associated_Drug_or_Biological_NDC_5,Program_Year,Recipient_ID


The fact that this filter returns an empty dataframe helps indicate that hospital and physician IDs are separate entities, and therefore the fact that the different types of IDs have overlaps is coincidental.

In [28]:
cms_edit1['Recipient_ID'] = cms_edit1['Recipient_ID'].astype(str)

In [29]:
def ID_addendum(col1,col2):
    if col1 == 'Covered Recipient Physician':
        return 'p'+col2
    else:
        return 'h'+col2

cms_edit1['Recipient_ID'] = cms_edit1.apply(lambda x: ID_addendum(x['Covered_Recipient_Type'],x['Recipient_ID']),axis=1)

In [30]:
cms_edit2 = cms_edit1
cms_edit2['Recipient_ID'].nunique()

615048

In [31]:
#Dropping additional columns, including ones about international geographical data
#After this, we should have 55 / 76 columns
cms_edit2 = cms_edit2.drop(
    ['Teaching_Hospital_ID',
     'Physician_Profile_ID',
    ],axis=1
)

In [32]:
#Dealing with null values by changing them to the string "Blank"
def impute_nan(df,col):
    df[col] = np.where(df[col].isnull(),'Blank',df[col])
    
for Columns in cms_edit2.columns:
    impute_nan(cms_edit2,Columns)
    
cms_edit2.head()

,Covered_Recipient_Type,Recipient_City,Recipient_State,Physician_Primary_Type,Physician_Specialty,Physician_License_State_code1,Physician_License_State_code2,Physician_License_State_code3,Physician_License_State_code4,Physician_License_State_code5,...,Product_Category_or_Therapeutic_Area_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Associated_Drug_or_Biological_NDC_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,Associated_Drug_or_Biological_NDC_5,Program_Year,Recipient_ID
0,Covered Recipient Physician,NORTH LITTLE ROCK,AR,Doctor of Dentistry,Dental Providers|Dentist|Endodontics,AR,Blank,Blank,Blank,Blank,...,Blank,Blank,Blank,Blank,Blank,Blank,Blank,Blank,2019,p827477.0
1,Covered Recipient Physician,CHAMPAIGN,IL,Doctor of Dentistry,Dental Providers|Dentist|Endodontics,IL,Blank,Blank,Blank,Blank,...,Blank,Blank,Blank,Blank,Blank,Blank,Blank,Blank,2019,p623090.0
2,Covered Recipient Physician,BROOKLYN,NY,Doctor of Dentistry,Dental Providers|Dentist,NY,Blank,Blank,Blank,Blank,...,Blank,Blank,Blank,Blank,Blank,Blank,Blank,Blank,2019,p3362255.0
3,Covered Recipient Physician,BROOKLYN,NY,Doctor of Dentistry,Dental Providers|Dentist,NY,Blank,Blank,Blank,Blank,...,Blank,Blank,Blank,Blank,Blank,Blank,Blank,Blank,2019,p3362255.0
4,Covered Recipient Physician,BROOKLYN,NY,Doctor of Dentistry,Dental Providers|Dentist,NY,Blank,Blank,Blank,Blank,...,Blank,Blank,Blank,Blank,Blank,Blank,Blank,Blank,2019,p3362255.0


## Preprocessing and Training

In [33]:
X_df = cms_edit2.drop(columns='Total_Amount_of_Payment_USDollars')
y_df = cms_edit2.Total_Amount_of_Payment_USDollars

In [37]:
#Now dealing with encoding? Binary or Hash > OneHot?
encoder_B = ce.BinaryEncoder()
encoder_B.fit_transform(X_df,y_df)

MemoryError: Unable to allocate 4.17 GiB for an array with shape (54, 10366845) and data type object

In [ ]:
len(X_df) * .7, len(X_df) * .3

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df,
                                                    test_size=0.3, 
                                                    random_state=1)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape

In [ ]:
# Prediction using the mean
train_mean = y_train.mean()
train_mean

In [ ]:
dumb_reg = DummyRegressor(strategy='mean')
dumb_reg.fit(X_train, y_train)
dumb_reg.constant_

In [ ]:
# Making arrays the length of the training / test sets respectively with only the training set mean as the value
y_tr_pred = dumb_reg.predict(X_train)
y_te_pred = train_mean * np.ones(len(y_test))

In [ ]:
# Baseline evaluation of performance using MSE
mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)

In [ ]:
# making decision tree
# dtree = tree.DecisionTreeRegressor()
# dtree.fit(X_train,y_train)

In [ ]:
# using model to make predictions?
#y_pred = dtree.predict(X_test)
#acc_DT = mean_squared_error(y_test,y_pred)

In [ ]:
#Visualize the tree?

In [ ]:
#Using a random forest model
#RFR = tree.RandomForestRegressor()
#RFR.fit(X_train,y_train)

In [ ]:
# using model to make predictions?
#y_pred = RFR.predict(X_test)
#acc_RFR = mean_squared_error(y_test,y_pred)

In [ ]:
#5-fold cross-validation?
#rf_default_cv_results = cross_validate(RF_pipe, X_train, y_train, cv=5)

The above shows that the input needed to be encoded as numerical values - the 

In [ ]:
#rf_cv_scores = rf_default_cv_results['test_score']
rf_cv_scores

In [ ]:
#n_est = [int(n) for n in np.logspace(start=1, stop=3, num=20)]
#grid_params = {
#        'randomforestregressor__n_estimators': n_est,
#        'standardscaler': [StandardScaler(), None],
#        'simpleimputer__strategy': ['mean', 'median']
#}
#grid_params